In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**#1.Import**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

**#2. 데이터 불러오기**

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
bureau = pd.read_csv('../input//home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')
pos_cash_balance = pd.read_csv('../input/home-credit-default-risk/POS_CASH_balance.csv')
previous_app = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
installments_payments = pd.read_csv('../input/home-credit-default-risk/installments_payments.csv')
credit_card_balance = pd.read_csv('../input/home-credit-default-risk/credit_card_balance.csv')

**#3. 1차 피처 엔지니어링**

In [ ]:
#bureau 테이블의 'SK_ID_CURR'를 기준으로 groupby, 과거 모든 대출 기록 count 하는 변수 생성
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()

In [ ]:
#bureau 특정 칼럼에 따라 집계값을 생성하는 함수를 정의
def normalize_categorical(df, group_var, col_name):
    
    # 카테고리 컬럼을 선택.
    categorical = pd.get_dummies(df.select_dtypes('object'))
    
    categorical[group_var] = df[group_var]
    
    # 그룹핑하는 칼럼에 따른 sum,mean값을 aggregation할 수 있도록 설정
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])                                              
    
    column_names = []
    
    for var in categorical.columns.levels[0]:
        for stat in ['count', 'count_norm']:
            column_names.append('%s_%s_%s' % (col_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [ ]:
bureau_counts = normalize_categorical(bureau, group_var = 'SK_ID_CURR', col_name = 'bureau')
bureau_counts.head()

In [ ]:
#데이터 그룹핑
#하나의 데이터셋에 merge할 수 있도록 'SK_ID_CURR'기준으로 그룹핑
data_bureau_agg=bureau.groupby(by='SK_ID_CURR').mean()
data_credit_card_balance_agg=credit_card_balance.groupby(by='SK_ID_CURR').mean()
data_previous_application_agg=previous_app.groupby(by='SK_ID_CURR').mean()
data_installments_payments_agg=installments_payments.groupby(by='SK_ID_CURR').mean()
data_POS_CASH_balance_agg=pos_cash_balance.groupby(by='SK_ID_CURR').mean()
data_bureau_agg.head()

+) 시각화해보기

In [ ]:
plt.figure(1)
plt.subplot(221)
app_train['CODE_GENDER'].value_counts(normalize=True).plot.bar(figsize=(20,10), title= 'Gender')

plt.subplot(222)
app_train['FLAG_OWN_CAR'].value_counts(normalize=True).plot.bar(title= 'Own Car?')

plt.subplot(223)
app_train['CNT_CHILDREN'].value_counts(normalize=True).plot.bar(title= 'Count Children')

plt.subplot(224)
app_train['FLAG_OWN_REALTY'].value_counts(normalize=True).plot.bar(figsize=(24,6), title= 'Has Realty?')



plt.show()

성별 중 여자의 수치가 높음
차가 없는 사람들 수치가 높음

In [ ]:
#데이터 병합
def merge(df):
    df = df.join(data_bureau_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    df = df.join(bureau_counts, on = 'SK_ID_CURR', how = 'left')
    df = df.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')
    df = df.join(data_credit_card_balance_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')    
    df = df.join(data_previous_application_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')   
    df = df.join(data_installments_payments_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    
    return df
train = merge(app_train)
test = merge(app_test)
display(train.head())

In [ ]:
print(train.shape, test.shape)

In [ ]:
#train, test 데이터 병합
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.TARGET.values
all_data = pd.concat([train, test]).reset_index(drop=True)
all_data.drop(['TARGET'], axis=1, inplace=True)

**#4.2차 피처 엔지니어링**

In [ ]:
#음수로 되어 있는 값은 절대값 처리
#'DAYS_BIRTH'는 365를 나누어줌
def correct_birth(df):
    
    df['DAYS_BIRTH'] = round((df['DAYS_BIRTH'] * (-1))/365)
    return df

In [ ]:
#절대값으로 나타내주기
def convert_abs(df):
    
    df['DAYS_EMPLOYED'] = abs(df['DAYS_EMPLOYED'])
    df['DAYS_REGISTRATION'] = abs(df['DAYS_REGISTRATION'])
    df['DAYS_ID_PUBLISH'] = abs(df['DAYS_ID_PUBLISH'])
    df['DAYS_LAST_PHONE_CHANGE'] = abs(df['DAYS_LAST_PHONE_CHANGE'])
    return df

In [ ]:
#missing value 들 0으로 채워주기
def missing(df):
    
    features = ['previous_loan_counts','NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_MEDI','OWN_CAR_AGE']
    
    for f in features:
        df[f] = df[f].fillna(0 )
    return df

In [ ]:
#위에서 처리해준 것들 all_data로 
def transform_app(df):
    df = correct_birth(df)
    df = convert_abs(df)
    df = missing(df)
    return df
all_data = transform_app(all_data)

In [ ]:
#고객들의 폰번호와 관련된 변수 생성 
all_data['NO_OF_CLIENT_PHONES'] = all_data['FLAG_MOBIL'] + all_data['FLAG_EMP_PHONE'] + all_data['FLAG_WORK_PHONE']
all_data.head()

In [ ]:
#고객들의 work_city와 live_city 관련된 변수 선언
all_data['FLAG_CLIENT_OUTSIDE_CITY'] = np.where((all_data['REG_CITY_NOT_WORK_CITY']==1) & (all_data['REG_CITY_NOT_LIVE_CITY']==1),1,0)
all_data['FLAG_CLIENT_OUTSIDE_REGION'] = np.where((all_data['REG_REGION_NOT_LIVE_REGION']==1) & (all_data['REG_REGION_NOT_WORK_REGION']==1),1,0)
all_data.head()

필요없는 칼럼 delete 해주기

In [ ]:
#위에서 생성한 변수들 중 쓸모 없는 칼럼 다시 제거
def delete(df):
     return df.drop(['FLAG_MOBIL', 'FLAG_EMP_PHONE' ,'FLAG_WORK_PHONE','REG_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION'], axis=1)
def transform(df):
    df = delete(df)
   
    return df
all_data = transform(all_data)
all_data.head()

In [ ]:
#모델링하는데 필요 없는 고유 식별 값(ex. ID) 제거
def delete_id(df):
    return df.drop(['SK_ID_CURR', 'SK_ID_PREV','SK_ID_BUREAU'], axis = 1)
all_data = delete_id(all_data)

In [ ]:
all_data.head()

In [ ]:
print(all_data.columns)

[missing values 처리]

In [ ]:
#숫자 빠진 것들
def miss_numerical(df):
    
    features = ['previous_loan_counts','NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_MEDI','OWN_CAR_AGE']
    numerical_features = all_data.select_dtypes(exclude = ["object"] ).columns
    #print(numerical_features)
    for f in numerical_features:
        #print(f)
        if f not in features:
            df[f] = df[f].fillna(df[f].median())
      
    return df

In [ ]:
def miss_categorical(df):
    
    categorical_features = all_data.select_dtypes(include = ["object"]).columns
    
    for f in categorical_features:
        df[f] = df[f].fillna(df[f].mode()[0])
        
    return df

In [ ]:
def transform_feature(df):
    df = miss_numerical(df)
    df = miss_categorical(df)
    #df = fill_cabin(df)
    return df

In [ ]:
all_data = transform_feature(all_data)


all_data.head()
        

[피처 스케일링]
수치형 데이터는 MinMaxScaler 활용

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def encoder(df):
    scaler = MinMaxScaler()
    numerical = all_data.select_dtypes(exclude = ["object"]).columns
    features_transform = pd.DataFrame(data= df)
    features_transform[numerical] = scaler.fit_transform(df[numerical])
    display(features_transform.head(n = 5))
    return df

all_data = encoder(all_data)


[인코딩]

In [ ]:

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0


# Iterate through the columns
for col in all_data:
    if all_data[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(all_data[col].unique())) <= 2:
            # Train on the training data
            le.fit(all_data[col])
            # Transform both training and testing data
            all_data[col] = le.transform(all_data[col])
            #test[col] = le.transform(test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
           
print('%d columns were label encoded.' % le_count)

In [ ]:
# dummy variables
all_data = pd.get_dummies(all_data)

display(all_data.shape)

+) 구글링 내용
:독립 변수끼리 상관 계수가 지나치게 높으면 다중공선성의 문제가 해결방법으로 특정 임계값을 정하여 그 값을 초과했을 때 해당 변수의 일부를 제거
=>엄지손가락의 규칙 : 상관 관계가 0.8을 초과한 경우 심각한 다중공선성 존재 가능으로 해석이 된다. 이 문서를 보고 임계값은 0.8로 정했고 그 임계값을 초과하는 경우는 변수에서 제외하는 방향으로 설정

In [ ]:
corrs = all_data.corr()
threshold = 0.8
above_threshold_vars = {}
for col in corrs:
    above_threshold_vars[col] = list(corrs.index[corrs[col]> threshold])
len(above_threshold_vars)

In [ ]:
cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []
for key, value in above_threshold_vars.items():
    cols_seen.append(key)
    for x in value :
        if x == key:
            next
        else:
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
cols_to_remove = list(set(cols_to_remove))

In [ ]:
all_data_removed = all_data.drop(columns=cols_to_remove)
all_data_removed.shape

**#5. Modeling**


In [ ]:
### Splitting features
train = all_data[:ntrain]
test = all_data[ntrain:]

print("Training shape", train.shape)
print("Testing shape", test.shape)

In [ ]:
from sklearn.model_selection import train_test_split

#data split
X_train, X_test, Y_train, Y_test = train_test_split(train, y_train, test_size = 0.3, random_state = 200)
print("X Training shape", X_train.shape)
print("X Testing shape", X_test.shape)
print("Y Training shape", Y_train.shape)
print("Y Testing shape", Y_test.shape)

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

logreg = LogisticRegression(random_state=0, class_weight='balanced', C=100)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict_proba(X_test)[:,1]
print('Train/Test split results:')
print("ROC",  roc_auc_score(Y_test, Y_pred))

In [ ]:
pred_test = logreg.predict_proba(test)
#print("ROC",  roc_auc_score(Y_test, pred_test))
submission = pd.read_csv('../input/home-credit-default-risk/sample_submission.csv')

submission['SK_ID_CURR']=app_test['SK_ID_CURR']
print(len(app_test['SK_ID_CURR']))
submission['TARGET']=pred_test
#converting to csv
#print(submission['TARGET'])
pd.DataFrame(submission, columns=['SK_ID_CURR','TARGET'],index=None).to_csv('homecreditada.csv')